In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

In [2]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')

In [5]:
test_df.isna().sum()

id                             0
Time_spent_Alone             425
Stage_fear                   598
Social_event_attendance      397
Going_outside                466
Drained_after_socializing    432
Friends_circle_size          350
Post_frequency               408
dtype: int64

In [6]:
df1=test_df.copy()

In [7]:
numerical_cols=list(df1.drop(columns=['id']).select_dtypes(include='number'))
categroical_cols=list(df1.drop(columns=['id']).select_dtypes('object'))
print(numerical_cols)
print(categroical_cols)

['Time_spent_Alone', 'Social_event_attendance', 'Going_outside', 'Friends_circle_size', 'Post_frequency']
['Stage_fear', 'Drained_after_socializing']


In [8]:
from sklearn.impute import SimpleImputer
si_mean=SimpleImputer(strategy='mean')
si_mode=SimpleImputer(strategy='most_frequent')

In [9]:
df1[numerical_cols]=si_mean.fit_transform(df1[numerical_cols])

In [10]:
df1[['Stage_fear','Drained_after_socializing']]=si_mode.fit_transform(df1[['Stage_fear','Drained_after_socializing']])

In [11]:
df1.isna().sum()

id                           0
Time_spent_Alone             0
Stage_fear                   0
Social_event_attendance      0
Going_outside                0
Drained_after_socializing    0
Friends_circle_size          0
Post_frequency               0
dtype: int64

In [12]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [13]:
for i in range(len(categroical_cols)):
    df1[categroical_cols[i]]=le.fit_transform(df1[categroical_cols[i]])

In [57]:
test_df.head(1)

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency
0,18524,3.0,No,7.0,4.0,No,6.0,NaN


In [58]:
df1.head(1)

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency
0,18524,3.0,0,7.0,4.0,0,6.0,5.028958


In [36]:
fin_df=df1[numerical_cols+categroical_cols]
fin_df.columns

Index(['Time_spent_Alone', 'Social_event_attendance', 'Going_outside',
       'Friends_circle_size', 'Post_frequency', 'Stage_fear',
       'Drained_after_socializing'],
      dtype='object')

In [19]:
clean_df=pd.read_csv('final.csv')

In [24]:
input_cols=list(clean_df.drop(columns=['Personality','Unnamed: 0','id']))
target_col='Personality'
print(input_cols)

['Time_spent_Alone', 'Stage_fear', 'Social_event_attendance', 'Going_outside', 'Drained_after_socializing', 'Friends_circle_size', 'Post_frequency']


In [25]:
x=clean_df[input_cols]
y=clean_df[target_col]

In [27]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(max_depth=12)
rf.fit(x,y)

,n_estimators,100
,criterion,'gini'
,max_depth,12
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [ ]:
x.head(1)

,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency
0,0.0,0,6.0,4.0,0,15.0,5.0


In [39]:
cols=list(x.columns)

In [40]:
fin_df_2=fin_df[cols]

In [43]:
fin_df_2.head(1)

,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency
0,3.0,0,7.0,4.0,0,6.0,5.028958


In [50]:
rf.predict(fin_df_2)

array([0, 1, 0, ..., 0, 0, 1], shape=(6175,))

In [54]:
dic={'id':test_df.id,'Personality':rf.predict(fin_df_2)}

In [67]:
submission_df=pd.DataFrame(dic)
submission_df

,id,Personality
0,18524,0
1,18525,1
2,18526,0
3,18527,0
4,18528,1
...,...,...
6170,24694,0
6171,24695,1
6172,24696,0
6173,24697,0


In [68]:
clean_df.head(1)

,Unnamed: 0,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0,0.0,0,6.0,4.0,0,15.0,5.0,0


In [69]:
train_df.head(1)

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,No,6.0,4.0,No,15.0,5.0,Extrovert


In [70]:
remap={0:'Extrovert',1:'Introvert'}

In [71]:
submission_df['Personality']=submission_df.Personality.map(remap)

In [75]:
submission_df.to_csv('submit.csv')

In [74]:
sample_submission=pd.read_csv('sample_submission.csv')
sample_submission

,id,Personality
0,18524,Extrovert
1,18525,Extrovert
2,18526,Extrovert
3,18527,Extrovert
4,18528,Extrovert
...,...,...
6170,24694,Extrovert
6171,24695,Extrovert
6172,24696,Extrovert
6173,24697,Extrovert
